In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

import os
batch_size = 64
epochs = 50
import keras
from PIL import Image
from keras.models import Model
# from keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers import Lambda, Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GaussianNoise, Input
from keras.layers import concatenate,GlobalAveragePooling2D, AveragePooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.datasets import mnist
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from keras.layers import LeakyReLU
from keras import backend as K


input_shape = (28,28,1)

# cnn_extractor = ResNet50(weights='imagenet',include_top=False, input_shape= input_shape)

def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    # conv_1 = Conv2D(filters_1x1, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1)

    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (3, 3), padding='same', activation='relu')(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same')(pool_proj)

    output = concatenate([x,conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)

    return output

input_layer = Input(shape=input_shape)

x = Conv2D(32, (5, 5), padding='same', strides=(1, 1), activation='relu')(input_layer)
x = Conv2D(32, (5, 5), padding='same', strides=(1, 1), activation='relu')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
x = Conv2D(64, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

x = Conv2D(128, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = Conv2D(128, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)

x = Dropout(0.5)(x)
x = Flatten()(x)

x = Dropout(0.5)(x)
x = LeakyReLU()(x)
x = Dense(1048)(x)
x = Dropout(0.5)(x)
x = LeakyReLU()(x)
x = Dense(10, activation='softmax')(x)

model = Model(input_layer, x)
model.summary()



Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        2112      
_________________________________________________________________
conv2d_4 (Conv2D)    

In [2]:

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape((x_train.shape[0],28,28,1))
x_test = x_test.reshape((x_test.shape[0],28,28,1))

# y_train = keras.utils.to_categorical(y_train, 10)
# y_test = keras.utils.to_categorical(y_test, 10)

print(x_train.shape)
print(y_train.shape)



(60000, 28, 28, 1)
(60000,)


In [3]:
# ada =keras.optimizers.Adadelta(lr=5.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1, validation_data=(x_test, y_test))
model.save_weights("mnist_2.h5")

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 35s 588us/step - loss: 2.3070 - acc: 0.1102 - val_loss: 2.3012 - val_acc: 0.1135
Epoch 2/50
59712/60000 [============================>.] - ETA: 0s - loss: 2.3019 - acc: 0.1097

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.savefig("accuracy.jpg")

plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.savefig("loss.jpg")
plt.show()